# HW9 Problem 7 Implementation

To address the weaknesses of rankings based solely on word counts, we explore the famous **pagerank** algorithm, which formed the basis for at least early versions of Google's search engine. Think of the whole internet as a directed graph where each node is a website, and there is a directed edge between node $i$ and $j$ if and only if website $i$ hyperlinks to website $j$. Intuitively, the pagerank algorithm seeks a ranking for which: 
- If a website is linked to by many other websites, then it's an important website
- If a website has only a few links, but those links come from authoritative sites (such as www.brown.edu), then it's also important.
- If a website links to a very large number of other websites, then the ``importance'' it transfers to each individual site is small. The pagerank algorithm uses Markov chains to allow the information provided by a link to implicitly flow both directions.

To illustrate pagerank, imagine a "random surfer" on the internet that starts at some webpage, and sequentially visits other webpages by following hyperlinks. As illustrated in Figure 1, the surfer chooses between the outgoing links from each page with equal probability. We can then define the "importance" of webpage $i$ as the long-term frequency with which this random surfer visits webpage $i$. If a node has $k$ outgoing edges, then the fraction of time a visit to this node is followed by each linked neighbor is only $\frac{1}{k}$. Denoting the state transition matrix by $T$, if the initial location of the surfer is uniform over the $m$ nodes so $\pi_{0} = [\frac{1}{m},\frac{1}{m},\dots, \frac{1}{m}]$, the probability of viewing each webpage after $n$ time steps is then $\pi_{0}T^{n}$. If $n$ is large and there are paths between all pairs of nodes, the state probabilities will converge to a stationary distribution $\pi = \pi T$. Sorting these probabilities gives the pagerank.


(a) Create the state transition matrix for the small network of Figure 1. What is the equilibrium distribution of this Markov chain? Which webpage has the highest pagerank?

In [1]:
import numpy as np

def equilibrium_distribution(P):
    """
    :param P: Transition Matrix
    :return: equilibrium distribution pi
    
    Note:
    1. I pi = P.T pi --> (I-P.T) pi = [0, 0, ..., 0].T
    2. constraint: np.ones pi = 1
    The first n equations have rank n-1, 
    we can directly replace the last row with constrain 2
    """
    
    # P is the transition matrix
    n = P.shape[0]  # Number of states
    # Create a matrix A by subtracting P from the identity matrix
    A = np.eye(n) - P.T
    # Add a constraint for the probabilities to sum to 1
    A[-1, :] = np.ones(n)
    # The target vector for the system of equations
    b = np.zeros(n)
    b[-1] = 1
    # Solve the system of linear equations
    pi = np.linalg.solve(A, b)
    return pi

In [3]:
P1 = np.array([[0, 1/3, 1/3, 1/3],
               [0, 0, 1/2, 1/2],
               [1, 0, 0, 0],
               [1/2, 0, 1/2, 0]])
pi_1 = equilibrium_distribution(P1)

print('Markov Equilibrium:', pi_1)
print('The first page has the highest importance')

Markov Equilibrium: [0.38709677 0.12903226 0.29032258 0.19354839]
The first page has the highest importance


Of course, the structure of real-world networks is more complex than Figure 1. A naive random surfer could get stuck in a "dead end" page (an absorbing state) or some locally connected subset of the full web. Thus, at each step, with probability $\alpha$ the surfer "teleports" to an arbitrary website at random; each of the $m$ websites has probability $\frac{1}{m}$ of being chosen. With probability $1-\alpha$, the surfer follows a hyperlink as above. The overall state transition matrix $G$ of this new Markov chain is then
\begin{equation}
    G = (1-\alpha) T + \alpha B, \qquad B = \frac{1}{m} \begin{bmatrix}
    1 & 1 & \dots & 1\\
    \vdots & \vdots & \ddots & \vdots\\
    1 & 1 & \dots & 1
    \end{bmatrix}
\end{equation}
The matrix $G$ is also known as the ``Google matrix''. In our experiments, we set $\alpha = 0.15$. \\
We now explore a larger network of $m = 9664$ websites. We provide the links structure of those sites in the matrix $L$, where $L(i,j) = 1$ if there is a link from website $i$ to website $j$, and $L(i,j) = 0$ otherwise. The **name** variable stores the names of each website.

(b) Write code that creates the state transition matrix $T$, and Google matrix $G$, for the provided data. If website i has no outgoing links, then T(i,i) = 1. You should double-check that for both T and G, the sum of the transition probabilities for each state equals $1$.

In [ ]:
import numpy as np

# Placeholder values for demonstration purposes
m = 9664  # number of websites
alpha = 0.15  # teleportation probability
L = np.random.randint(0, 2, (m, m))  # random link structure matrix
name = np.array(["Website " + str(i) for i in range(m)])  # website names

# Create the transition matrix T
T = np.zeros((m, m))

# Populate the T matrix
for i in range(m):
    if np.sum(L[i, :]) == 0:
        # If website i has no outgoing links, set T(i,i) = 1
        T[i, i] = 1
    else:
        # Distribute transition probabilities equally among outgoing links
        T[i, :] = L[i, :] / np.sum(L[i, :])

# Check if the sum of the transition probabilities for each state equals 1
assert np.allclose(T.sum(axis=1), 1), "Sum of probabilities in T is not 1"

# Create matrix B
B = np.ones((m, m)) / m

# Create the Google matrix G
G = (1 - alpha) * T + alpha * B

# Check if the sum of the transition probabilities for each state equals 1 in G
assert np.allclose(G.sum(axis=1), 1), "Sum of probabilities in G is not 1"

# Returning the first few entries of T and G for inspection
T[:5, :5], G[:5, :5]
